---
title: "Quick Start of MAAS"
output: html_document
---

In [ ]:
knitr::opts_chunk$set(echo = TRUE, warning = FALSE, message = TRUE)

In [ ]:
library(MAAS)
library(tidydr)
library(withr)

# Set seed for reproducibility
set.seed(1234)
# Load example data
data("maas_example")
# Run MAAS
maas.test <- MAAS(maas_example$Peak, maas_example$CNV, maas_example$SNV, dims = 2:5)

In [ ]:
# Evaluate clustering performance
clusPerformance <- data.frame(matrix(nrow = length(maas.test)-1, ncol = 5),
                              row.names = paste0("dims=", 2:length(maas.test)))
colnames(clusPerformance) <- paste0("k=", 2:6)

for(i in 1:(length(maas.test)-1)){
  for(j in 2:6){
    df <- as.data.frame(maas.test[[i]]$W)
    maas.tmp.clu <- with_seed(2, kmeans(df, centers = j)$cluster)
    clusPerformance[i,j-1] <- clusteringMetric(maas.test[[i]]$W, clu = maas.tmp.clu, disMethod = "cosine")
  }
}
clusPerformance

In [ ]:
# Select one layer and perform clustering
df <- as.data.frame(maas.test[[1]]$W)
maas.clu <- data.frame(Cluster = with_seed(2, kmeans(df, centers = 2)$cluster))
maas.clu$Cluster <- as.factor(maas.clu$Cluster)

# Compute UMAP
umap.axis <- with_seed(2, umap(df, n_neighbors = 10, metric = "correlation"))
umap.axis <- as.data.frame(umap.axis)
umap.axis$Cluster <- maas.clu$Cluster
colnames(umap.axis) <- c("UMAP-1", "UMAP-2", "Cluster")

# UMAP plot
ggplot(umap.axis, aes(`UMAP-1`, `UMAP-2`)) +
  geom_point(aes(color = Cluster), size = 1.75) +
  theme_dr()+
  labs(x = "UMAP-1", y = "UMAP-2")+
  theme(panel.grid = element_blank(),
        axis.title = element_text(size = 14),
        axis.text = element_blank(),
        legend.text = element_text(size = 12))

```{r session}
sessionInfo()
```